In [ ]:
import os
import pandas as pd
from pathlib import Path

In [ ]:
MUT_PATH = '/home/yutianc/muninn_sc2/inputs/escape_1000/mutations.tsv'

IN_DIR = '/home/yutianc/bjorn_rep/data/sc2'
OUT_DIR = '/home/yutianc/muninn_sc2/inputs/escape_1000'

FASTA_DIR = '/home/yutianc/bjorn_rep/data/sc2/consensus_sequences'

In [ ]:
mut = pd.read_csv(MUT_PATH, sep='\t')
dms = pd.read_csv(os.path.join(IN_DIR, "final_variant_scores_BA.1_BA.2.txt"), sep=',', header=0)
metadata = pd.read_csv(os.path.join(IN_DIR, "metadata.csv"))
lineage = pd.read_csv(os.path.join(IN_DIR, "lineage_report.csv"))
evescape = pd.read_csv(os.path.join(IN_DIR, "full_spike_evescape.csv"))

# the unique samples in the subset, will be used to subset other files
sra = list(mut["sra"].unique())

## DMS Data

In [ ]:
# Check if Hu-1_v1 and Hu-1_v2 are the same thing..
dms_v1 = dms[dms['target'] == 'Wuhan-Hu-1_v1']
dms_v2 = dms[dms['target'] == 'Wuhan-Hu-1_v2']

m = pd.merge(dms_v1, dms_v2, how="outer", on=["position"], indicator=True)
assert len(m[m["wildtype_x"] != m['wildtype_y']]) == 0
assert len(m[m["_merge"] != 'both']) == 0

# they are the same thing, so I will treat both of them as Hu-1

In [ ]:
dms = pd.read_csv(os.path.join(IN_DIR, "final_variant_scores_BA.1_BA.2.txt"), sep=',', header=0)

ref_map = {'Omicron_BA1': 'NC_045512.2_escape_BA.1_rbd',
           'Omicron_BA2': 'NC_045512.2_escape_BA.2_rbd',
           'Wuhan-Hu-1_v1': 'NC_045512.2',
           'Wuhan-Hu-1_v2': 'NC_045512.2',
           'Beta': 'Beta',
           'Alpha': 'Alpha',
           'Delta': 'Delta',
           }

# this is rbd region only, so YP.. is used here
dms['GFF_FEATURE'] = 'YP_009724390.1_' + dms['target'].map(ref_map)

# check if gffs are correct
mut['mutation'] = mut['ref_aa'] + str(mut['pos_aa']) + mut['alt_aa']
merged = pd.merge(dms, mut, how='inner', on=['mutation'], indicator=True)

assert not (
    (merged["_merge"] == "both") &
    (merged["GFF_FEATURE_x"] != merged["GFF_FEATURE_y"])
).any()


# currently only for these ref genomes
dms = dms[dms['target'].isin(['Omicron_BA1', 'Omicron_BA2', 'Wuhan-Hu-1_v1', 'Wuhan-Hu-1_v2'])]

dms.to_csv(os.path.join(OUT_DIR, "dms.tsv"), sep='\t', index=False)

## Metadata

In [ ]:
metadata["mut_sra"] = metadata['fasta_hdr'].apply(lambda x: str(x).split("/")[2] if len(str(x).split("/")) == 4 else x)

# for those id and fasta_hdr mismatch, overwrite fasta_hdr with id
mask = metadata["mut_sra"].notna() & (metadata["mut_sra"] != metadata["ID"])
metadata.loc[mask, "ID"] = metadata.loc[mask, "mut_sra"]

metadata_sampled = metadata[metadata["mut_sra"].isin(sra)]
metadata_sampled = metadata_sampled.drop(columns={"mut_sra"})
metadata = metadata.drop(columns={"mut_sra"})

metadata_sampled.to_csv(os.path.join(OUT_DIR, "metadata_sampled.tsv"), sep="\t", index=False)
metadata.to_csv(os.path.join(IN_DIR, "metadata_cleaned.csv"), index=False)


## Lineage

In [ ]:
def fasta_first_seq_id(path: Path):
    with path.open("r", encoding="utf-8", errors="replace") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                seq_id = line[1:].split()[0]
                seq_id = seq_id.split("/")[2] if len(seq_id.split("/")) ==  4 else seq_id
                return seq_id
            return None
    return None 

def find_fname_sid_mapping(dir_path: str, exts=(".fasta")):
    d = Path(dir_path)
    fname, sid = [], []

    for p in d.iterdir():  # no subdirs
        if exts and p.suffix.lower() not in exts:
            continue

        seq_id = fasta_first_seq_id(p)

        fname.append(p.stem)
        sid.append(seq_id)

    return fname, sid


In [ ]:
# similarily, overwrite the mismatched taxon with id from fasta file
fname, sid = find_fname_sid_mapping(FASTA_DIR)
dic_lineage = dict(zip(fname, sid))
lineage["taxon"] = lineage["taxon"].map(dic_lineage)

lineage_sampled = lineage[lineage["taxon"].isin(sra)]

lineage_sampled.to_csv(os.path.join(OUT_DIR, "lineage_sampled.csv"), index=False)
lineage.to_csv(os.path.join(IN_DIR, "lineage_cleaned.csv"), index=False)

## EVEscape

In [ ]:
# evescape is Hu-1 only
# and this file is for spike only, so the corresponding gff is YP_009724390.1

evescape["GFF_FEATURE"] = 'YP_009724390.1_NC_045512.2'
evescape["mutation"] = evescape["wt"] + evescape["i"] + evescape["mut"]
# check if GFFs match with the ones in mut file
merged = pd.merge(mut, evescape, on="mutation", how="inner")

assert (merged["GFF_FEATURE_x"] == merged["GFF_FEATURE_y"]).all()

evescape = evescape.drop(columns={"mutation"})
evescape.to_csv(os.path.join(OUT_DIR, "evescape.tsv"), sep='\t', index=False)